In [1]:
import csv
import pandas

In [2]:
comma_data = []
with open("semeval-2018-task4/dat/friends.train.scene_delim.conll") as fp:
    for line in list(fp):
        #print(repr(line))
        new = line.split()
        #print(new)
        comma_data.append(new)

In [389]:
pronoun_list = ['he', 'she', 'it', 'they', 'him', 'her', 'them', 'his', 'her', 'hers', 'its', 'their', 'theirs']
ignored_references = ['i', 'me', 'you', 'my', 'mine', 'myself', 'your', 'yourself', 'yours']
relations = ['brother', 'sister', 'sibling', 'mother', 'father', 'child', 'friend', 'daddy', 'mom', 'dad', 'boyfriend', 'girlfriend','grandmother','grandfather', 'son', 'daughter', 'aunt', 'uncle', 'date', 'wife', 'husband', 'partner']
#nickname or other name for the person > e.g. boy for birthday boy. These seem to be mostly for talking to someone directly, not about another person. So maybe ignore these?
nickname = ['sweetie', 'guy', 'woman', 'boy', 'man', 'girl', 'honey', 'pal', 'one', 'kid','baby','princess', 'queen','bitch']
names = ['Joey','Rachel','Monica','Ross','Phoebe','Chandler']
last_names = ['Tribbiani', 'Green', 'Geller', 'Buffay', 'Bing', 'Willick', 'Bunch', 'Goodacre', 'Farber']
reference_types = ['F', 'R', 'P', 'S', 'A']

In [259]:
def split_data_per_sentence(data):
    sentences = []
    one_sentence = []
    for lst in data:
        if not lst:
            one_sentence.append(lst)
            sentences.append(one_sentence)
            one_sentence = []
        else:
            one_sentence.append(lst)
    return sentences

def find_identifiers_per_token(data, identifier, person, speakers=['Monica_Geller', 'Joey_Tribbiani', 'Phoebe_Buffay', 'Rachel_Green', 'Chandler_Bing', 'Ross_Geller']):
    '''
    Assigns reference types to each token for the given identifier and person (should represent the same person).
    '''
    token_patterns = []
    for lst in data:
        if not lst:
            token_patterns.append('new')
        elif lst[-2] == "part" or lst[-2] == 'document':
            token_patterns.append('new_scene')
        elif lst[-1] != '-' and lst[-1] != '(DATE)':   #add more when I find more > person, organization, etc.? or this is [-2] instead of [-1]
            if lst[-2] != 'part' and lst[-2] != 'document':
                if lst[-1] == identifier[:-1]:
                    if lst[3] in names or lst[3] == person:
                        token_patterns.append('A')
                elif lst[-1] == identifier:
                    if lst[-3] in speakers:
                        if lst[3].lower() not in ignored_references:
                            if lst[3] in pronoun_list:
                                token_patterns.append('P')
                            elif lst[3].lower() in relations:
                                token_patterns.append('R')
                            elif lst[3] in names or lst[3] == person:
                                token_patterns.append('F')
                            elif lst[3] in last_names:
                                token_patterns.append('S')
                            else:
                                token_patterns.append('.')
                        else:
                            token_patterns.append('.')
                    else:
                        token_patterns.append('.')
                else:
                    token_patterns.append('.')
            else:
                pass
        else:
            token_patterns.append('.')
    return token_patterns

def into_sentences(data):
    '''
    Outputs the sentences as lists of tokens.
    '''
    sentences = []
    one_sentence = []
    for lst in data:
        for lst2 in lst:
            if lst2 and lst2[0] != '#begin' and lst2[0] != '#end':
                one_sentence.append(lst2[3])
        sentences.append(one_sentence)
        one_sentence = []
    return sentences

def list_into_string_lists(lst, split):
    '''
    To turn one list with each token being one element into a list of elements that are strings. To make the patterns.
    '''
    string = ''.join([str(elem) for elem in lst])
    splitted = string.split(split)
    return splitted

def reference_per_utterance(lst):
    """
    If there exists a reference type in an utterance, keep only those. Utterances without reference type become a period.
    """
    ref_per_utterance = []
    for pattern in lst:
        pattern_contains_ref = any(reference_type in pattern for reference_type in reference_types)
        #print(pattern_contains_ref)
        if not pattern_contains_ref:
            ref_per_utterance.append('.')
        else:
            ref_per_utterance.append(pattern.replace('.',''))
    
    return ref_per_utterance

In [534]:
test_list = find_identifiers_per_token(comma_data, identifier='(248)', person="Monica")
scenes = list_into_string_lists(test_list, 'new_scene')

list_of_scene_utterances = []
for scene in scenes:
    test = list_into_string_lists(scene, 'new')
    list_of_scene_utterances.append(test)
    
#print(list_of_scene_utterances)
    
utterance_patterns_per_scene = []
for lst in list_of_scene_utterances:
    test2 = reference_per_utterance(lst)
    utterance_patterns_per_scene.append(test2)
    
#print(utterance_patterns_per_scene[1:])

string_utterance_patterns_per_scene = []
for scene in utterance_patterns_per_scene:
    test3 = ''.join(scene)
    string_utterance_patterns_per_scene.append(test3)
    
#print(string_utterance_patterns_per_scene[1:])

In [522]:
#human annotated and generated patterns that were manually found
human_annotated = ['FPR........F', 'FF........', '.APRP..FPPF....', '..FPR....A..', '...RF......', '..F....F..', '....FR.....', '.....RF....', '......RF...', '.......FR..', '.......AFR.', 'RF..F......', '.A.....FF..', '..RF.......', '...ARF.....', '...RF......', '....A.FP...', '.....FRF...', '.......APP..', 'AR', 'FR', 'AP', 'FP.F', 'AF.....FF', 'AR', 'RFF...FR', 'RF', 'APR', 'APP', 'FFR', 'AAR', 'RF', 'FPR', 'RFPFR', 'F.F', 'AR.....F', 'RFPFR', 'AR', 'A.F', 'AFFR.F', 'A........F', 'FFR', 'RFFR..RF', 'AP.F', 'F.F', 'RF', 'RAF..F', 'RF', 'FR...FR', 'AR..FR', 'A.....F', 'FR', 'RFPA', 'RF....F', 'RF', 'APP', 'AR', 'A..F', 'FR.FP.F..A', 'R.....F', 'RA...F', 'AP', 'A.......FFR', 'AR..F', 'A.....F', 'APPP', 'FR', 'A..F', 'APRP...F', 'FPR..F', 'FPR....FPR', 'RFF', 'AR']
generated = ['FFR........AFR', '.F..AFRFFR....', '..AFR....AFR..', '....F.A...', 'AA', 'AFR', 'FFR', '.A......FF', '.AFR......A.', 'FFR...A', 'FF.....A', 'AFR', 'FFRFFR', 'AFRPFR', 'FFR', 'AFR', 'PFR.FF', 'FFR......FFR', 'AAFRA...AFR', 'A..F', 'AFR', 'AA..A...AF', 'FFF', 'AFR', 'AFR', 'PFR....PFR', 'AFF...A', 'AFR.FFR', 'FA.A', 'FAFR.FFR', 'AFR........AFR', 'FFR', 'A..A..FF', 'FFR.F', 'FFR', 'FFR', 'F.PF', 'FFA..A', 'FFR..AFR', 'FF...FR', 'F..FFR', 'FFR.....F', 'PFR', 'PFR', 'AFR', 'AFR', 'PFR', 'AFRAFAFR', 'AFR', 'F..A', 'A.......AFR', 'AAA', 'FFR', 'FFR.A..FFR..A', 'A.PFR...FFR', 'FFR.....AFR', 'A.....F', 'FFRPFR', 'FF', 'AA', 'PFR', 'PFR.......FFR', 'FFR', 'AAFR', 'FA..FFR', 'F.....FFR', 'FFR', 'FFRAFR', 'A.....A', 'FFR..A..A', 'A..PFR..FFR', 'AFRF...F', 'F....AFR', 'FFR', 'FA', 'FFR']

In [396]:
#will find patterns in the text (so for .....P...P..FF....). Find the mean values? input should be utterance_patterns_per_scene
def find_patterns(data, first_compare, second_compare):
    '''
    Calculates the distance between the first reference type and the second reference type. 
    '''
    distances = []
    for scene in data:
        distance = 0
        for token in scene:
            if token == first_compare and distance == 0:
                distances.append('start')
            elif not distances:
                continue
            elif token == "." and distances[-1] == 'start':
                distance+=1
            elif token == second_compare and distances[-1] == 'start':
                distance+=1
                distances.append(distance)
                distance = 0
                if first_compare == second_compare:
                    distances.append('start')

    distances = [x for x in distances if x != 'start']
    return distances

In [397]:
#example of the distance between R and P, as well as how often the combation occurs
find_patterns(string_utterance_patterns_per_scene, "R", "P")

[70, 2]

In [523]:
def find_non_empty_patterns(pattern_data):
    '''
    Find patterns that don't have a zero value as the frequency.
    '''
    first = second = ["P", "F", "A", "S", "R"]
    not_empty = []
    for i in first:
        for j in second:
            lst = find_patterns(pattern_data, i, j)
            if not lst:
                pass
            else:
                not_empty.append([i,j])
    return not_empty

#non_empty_lists = find_non_empty_patterns(string_utterance_patterns_per_scene)
#non_empty_lists = find_non_empty_patterns(human_annotated)
non_empty_lists = find_non_empty_patterns(generated)

In [524]:
import statistics

def get_mean(non_empty_list, pattern_list):
    for i in non_empty_list:
        print(i)
        print(statistics.mean(find_patterns(pattern_list, i[0], i[1])))
        print(len(find_patterns(pattern_list, i[0], i[1])))
              
#get_mean(non_empty_lists, string_utterance_patterns_per_scene)
#get_mean(non_empty_lists, human_annotated)
get_mean(non_empty_lists, generated)

['P', 'P']
3
4
['P', 'F']
1
13
['P', 'A']
1.5
10
['P', 'R']
1
13
['F', 'P']
1.6153846153846154
13
['F', 'F']
4.4324324324324325
37
['F', 'A']
2.36734693877551
49
['F', 'R']
1.3376623376623376
77
['A', 'P']
1.4
10
['A', 'F']
1.7959183673469388
49
['A', 'A']
4.2592592592592595
27
['A', 'R']
1.4285714285714286
42
['R', 'P']
1.6153846153846154
13
['R', 'F']
2.276315789473684
76
['R', 'A']
2.380952380952381
42
['R', 'R']
4.56
25


In [535]:
# print(find_patterns(utterance_patterns_per_scene, "P", "S"))
#print(comma_data[0:50])

In [538]:
def find_vague_references(data, identifiers=['(248)', '(355)', '(183)', '(292)', '(59)', '(306)', '(271)'], persons=['Monica', 'Joey','Rachel','Phoebe','Chandler','Ross','Judy'], speakers=['Monica_Geller', 'Joey_Tribbiani', 'Phoebe_Buffay', 'Rachel_Green', 'Chandler_Bing', 'Ross_Geller']):
    '''
    Find vagueness counts. If an utterance contains a reference to a person without it being their name or one of the 
    ignored identifiers, it will add that.
    '''
    #wat moet: als de identifier een van de friends is en niet de naam is, kijk of zijn/haar naam wordt eerder in die zin wordt genoemd.
    sentence = []
    number_of_sentences=0
    list_number_of_sentences=[]
    vagueness=0
    vagueness_list = []
    for lst in data:
        if not lst:
            sentence=[]
            number_of_sentences+=1
        elif lst[-2] == "part" or lst[-2] == 'document':
            vagueness_list.append(vagueness)
            vagueness=0
            list_number_of_sentences.append(number_of_sentences)
            number_of_sentences=0
        elif lst[-1] != '-' and lst[-1] != '(DATE)':   #add more when I find more > person, organization, etc.? or this is [-2] instead of [-1]
            if lst[-2] != 'part' and lst[-2] != 'document':
                for identifier in identifiers:
                    if lst[-1] == identifier[:-1]:
                        if lst[3] in names or lst[3] in persons:
                            pass
                    elif lst[-1] == identifier:
                        if lst[-3] in speakers:
                            if lst[3].lower() not in ignored_references:
                                if lst[3] in names or lst[3] in persons:
                                    pass
                                else:
                                    if lst[3] not in sentence:
                                        vagueness += 1
                                        sentence.append(lst[3])
                                        #print(sentence)
            else:
                pass

    return vagueness_list, list_number_of_sentences

In [539]:
#frequency of a vague utterance and how many sentences there are in total
vagueness,sentences=find_vague_references(comma_data[1:])

In [469]:
vagueness_score = [int(vague) / int(sentence) for vague,sentence in zip(vagueness, sentences)]
print(vagueness_score)

[0.021052631578947368, 0.08411214953271028, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08571428571428572, 0.0, 0.0, 0.0, 0.060240963855421686, 0.018518518518518517, 0.0, 0.0, 0.0, 0.0, 0.07894736842105263, 0.037037037037037035, 0.0, 0.028169014084507043, 0.0, 0.0, 0.12121212121212122, 0.0, 0.21052631578947367, 0.03225806451612903, 0.0, 0.0, 0.06451612903225806, 0.02702702702702703, 0.01818181818181818, 0.043478260869565216, 0.0, 0.043478260869565216, 0.047619047619047616, 0.0, 0.0, 0.061224489795918366, 0.07692307692307693, 0.0, 0.0, 0.07692307692307693, 0.025974025974025976, 0.0, 0.09090909090909091, 0.09375, 0.12727272727272726, 0.08333333333333333, 0.21739130434782608, 0.04, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.06976744186046512, 0.029411764705882353, 0.0, 0.0, 0.04, 0.038461538461538464, 0.0, 0.029850746268656716, 0.0, 0.0625, 0.13793103448275862, 0.0, 0.0, 0.0, 0.0, 0.015873015873015872, 0.018867924528301886, 0.0, 0.0, 0.0, 0.0, 0.01639344262295082, 0.03125, 0.08928571428571429, 0.060606060

In [470]:
#vagueness score
sum(vagueness_score)/len(vagueness_score)

0.04673370851759234

In [540]:
def find_MOA_scores(data, identifiers=['(248)', '(355)', '(183)', '(292)', '(59)', '(306)', '(271)'], persons=['Monica', 'Joey','Rachel','Phoebe','Chandler','Ross','Judy'], speakers=['Monica_Geller', 'Joey_Tribbiani', 'Phoebe_Buffay', 'Rachel_Green', 'Chandler_Bing', 'Ross_Geller']):
    '''
    Finds the MOA score per utterance. Per scene, I create a list of persons that have been mentioned.
    For every ambiguous reference, it looks at how many persons have been mentioned before that.
    '''
    #wat moet: voor elke pronoun, kijk hoeveel personen daarvoor zijn gereferenced (lastig omdat er geen identifier is voor wie man en vrouw is, dus ik doe alleen de 6 friends
    sentence = []
    number_of_sentences=0
    list_number_of_sentences=[]
    vagueness=0
    vagueness_list = []
    
    mentioned = 0
    mentioned_list = []
    MOA_list = []
    MOA_score = 0
    count = 0
    for lst in data:
        if not lst:
            sentence=[]
            number_of_sentences+=1
        elif lst[-2] == "part" or lst[-1] == 'document':
            MOA_list.append(MOA_score)
            mentioned=0
            mentioned_list = []
            list_number_of_sentences.append(number_of_sentences)
            number_of_sentences=0
        elif lst[-1] != '-' and lst[-1] != '(DATE)':   #add more when I find more > person, organization, etc.? or this is [-2] instead of [-1]
            if lst[-2] != 'part' and lst[-2] != 'document':
                if lst[-3] in speakers:
                    if lst[-2] == '(PERSON)':
                        if lst[-1] in mentioned_list:
                            pass
                        else:
                            mentioned+=1
                            mentioned_list.append(lst[-1])
                            #print(mentioned_list)
                    
                    else:
                            if lst[3].lower() == 'she' or lst[3].lower() == 'her' or lst[3].lower() == 'he' or lst[3].lower() == 'his':
                                MOA_score = mentioned
                                count +=1
            else:
                pass

    #print(mentioned_list)
    return MOA_list, list_number_of_sentences

In [541]:
MOA,sentences=find_MOA_scores(comma_data[:])

In [542]:
print(MOA)

[0, 3, 3, 5, 5, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 0, 0, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 1, 1, 1, 1, 1, 1, 4, 4, 1, 1, 1, 1, 4, 4, 4, 4, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 0, 0, 0, 0, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 0, 0, 4, 4, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 4, 4, 0, 0, 3, 3, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 3, 3, 3, 3, 6, 6, 6, 6, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 6, 6, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 2, 2, 4, 4, 0, 0, 2, 2, 2, 2, 2, 2, 5, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 0, 0, 0, 0, 4, 4, 4, 4, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 3, 3, 3, 3, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 

In [510]:
from collections import Counter
Counter(MOA)

Counter({0: 139, 3: 106, 5: 36, 2: 136, 4: 77, 1: 240, 6: 10, 7: 2, 8: 2})

In [511]:
print((3*106)+(5*36)+(2*136)+(4*77)+240+60+14+16)

1408


In [543]:
print(sentences[1:-1:2])

[95, 107, 20, 15, 27, 27, 14, 70, 15, 18, 37, 83, 54, 27, 19, 44, 31, 38, 81, 9, 71, 24, 26, 33, 20, 19, 93, 21, 22, 31, 37, 55, 23, 18, 23, 21, 14, 36, 49, 13, 9, 31, 13, 77, 83, 33, 32, 55, 48, 23, 100, 42, 19, 37, 17, 50, 43, 34, 32, 28, 25, 78, 14, 67, 30, 32, 29, 57, 21, 23, 22, 63, 53, 32, 59, 33, 24, 61, 64, 56, 33, 64, 46, 39, 45, 22, 14, 25, 26, 19, 41, 28, 71, 51, 24, 30, 21, 47, 25, 22, 27, 33, 48, 30, 12, 33, 36, 58, 64, 55, 103, 19, 89, 54, 30, 22, 46, 38, 42, 55, 24, 25, 20, 51, 39, 19, 38, 28, 88, 46, 49, 16, 22, 79, 86, 16, 57, 38, 20, 24, 70, 15, 47, 23, 21, 71, 30, 28, 78, 47, 51, 42, 52, 30, 22, 26, 20, 52, 12, 45, 42, 22, 26, 32, 33, 25, 59, 29, 21, 72, 56, 22, 73, 20, 34, 28, 38, 49, 45, 33, 41, 21, 11, 20, 11, 26, 37, 48, 38, 24, 42, 25, 27, 10, 9, 34, 60, 33, 58, 39, 28, 32, 35, 25, 28, 17, 46, 37, 26, 28, 16, 47, 33, 55, 26, 16, 31, 31, 25, 40, 73, 106, 90, 27, 79, 21, 29, 31, 36, 70, 25, 13, 53, 81, 22, 63, 38, 14, 41, 22, 46, 30, 68, 43, 35, 33, 28, 39, 51, 69

In [544]:
sum(sentences[1:-1:2])/len(sentences[1:-1:2])

36.21447721179624

In [545]:
print(sum(sentences))

13556
